In [6]:
%pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 18.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.document_loaders import PyMuPDFLoader

# Load a PDF
loader = PyMuPDFLoader("examle_file.pdf")
documents = loader.load()


In [9]:
import numpy as np
import pandas as pd
import nltk
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
nltk.download('punkt')
import os
import re
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/konstantinosdalampekis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
def ocr_from_image(image_path):
    text = pytesseract.image_to_string(image_path)
    return text


def extract_text_from_pdf(file_path):
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    except Exception as e:
        print(f"Error with PDFplumber: {e}. Attempting OCR.")
        images = convert_from_path(file_path)
        for image in images:
            text += ocr_from_image(image)
    return text

def parse_pdf(file_path):
    print(f"Parsing {file_path}")
    parsed_text = extract_text_from_pdf(file_path)
    print(parsed_text)
    return parsed_text

In [11]:
import os
import PyPDF2

def parse_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text

In [ ]:
output_folder = './parsed_docs'
os.makedirs(output_folder, exist_ok=True)

pdf_folder = './pdf_files'
for filename in os.listdir(pdf_folder):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_folder, filename)
        parsed_text = parse_pdf(file_path)
        output_path = os.path.join(output_folder, filename.replace('.pdf', '.txt'))
        with open(output_path, 'w', encoding='utf-8') as output_file:
            output_file.write(parsed_text)
        print(f'Parsed and saved: {output_path}')

print('All PDF files have been processed and stored in the parsed_docs folder.')

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import os

def load_and_chunk(file_path, chunk_size=500, chunk_overlap=100):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    chunks = splitter.create_documents([text])
    return chunks  # Each chunk is a Document object with .page_content

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def generate_embeddings(chunks, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    texts = [chunk.page_content for chunk in chunks]
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy(), texts

In [7]:
def save_chunks(chunks, output_path='rag_chunks.txt'):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, chunk in enumerate(chunks):
            f.write(f"Chunk {i + 1}:\n")
            f.write(chunk.page_content + "\n\n")
    print(f"Chunks saved to {output_path}")

In [35]:
file_path = 'your_file_path.txt'  # or preprocessed PDF-to-text
chunks = load_and_chunk(file_path)

embeddings, texts = generate_embeddings(chunks)


In [17]:
import elasticsearch
from elasticsearch import Elasticsearch

In [18]:
ELASTICSEARCH_HOST = "https://your_elasticsearch_url"
ELASTICSEARCH_PORT = 0****
ELASTICSEARCH_USERNAME = "your_username"  # Replace with actual username
ELASTICSEARCH_PASSWORD = "your_password"  # Replace with actual password

es_client = Elasticsearch(
    f"{ELASTICSEARCH_HOST}",
    basic_auth=(ELASTICSEARCH_USERNAME, ELASTICSEARCH_PASSWORD),
    verify_certs=False,
    request_timeout=60  # ⏰ increase timeout

)

es_client.ping()

/opt/homebrew/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ae9a1c3a-a273-4962-8dfb-37c3f360d8db.br37s45d0p54n73ffbr0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [36]:
from langchain_community.vectorstores import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = ElasticsearchStore.from_texts(
    texts=texts,
    embedding=embedding_model,
    index_name="your_index_name",
    es_connection=es_client  # ✅ Use es_connection
)


/opt/homebrew/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ae9a1c3a-a273-4962-8dfb-37c3f360d8db.br37s45d0p54n73ffbr0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ae9a1c3a-a273-4962-8dfb-37c3f360d8db.br37s45d0p54n73ffbr0.databases.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [112]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # You can tune k
)


In [110]:
es_client.ping()  # Returns True?


True

In [ ]:
query = "Tάσεις που να αφορούν ειδικά κοινά ταξιδιωτών"

# Retrieve top-k relevant chunks
results = retriever.get_relevant_documents(query)

# Print the matched content
for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}\n")


Elastic Search Retriever

In [ ]:

query = "Tάσεις που να αφορούν ειδικά κοινά ταξιδιωτών"






In [97]:
from sentence_transformers import SentenceTransformer

# Use the same model you used for indexing
model = SentenceTransformer('all-MiniLM-L6-v2')
query_vector = model.encode(query).tolist()


In [ ]:
# Step 2: Run the kNN search
response = es_client.search(
    index="your_index_name",
    knn={
        "field": "vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    }
)

# Step 3: Display results
for i, hit in enumerate(response["hits"]["hits"]):
    print(f"\n🔹 Document {i+1} (score: {hit['_score']}):")
    print(hit["_source"].get("text", "[No text field]"))

In [118]:
print(query_vector)

[-0.04851391166448593, 0.08879464119672775, 0.04345763474702835, -0.02750181220471859, -0.04511796683073044, -0.0488043911755085, 0.054508406668901443, 0.022584620863199234, 0.011165088973939419, 0.05132095143198967, -0.0015676941256970167, -0.01733952946960926, -0.05016481503844261, 0.006822132971137762, -0.08699511736631393, -0.09649045020341873, -0.05166640505194664, 0.07538685947656631, -0.008570750243961811, -0.03920086473226547, 0.04275991767644882, -0.03393405303359032, 0.06912093609571457, 0.011319302953779697, 0.020726162940263748, 0.07011815905570984, -0.018094802275300026, -0.1178569421172142, 0.06687517464160919, 0.03929853066802025, -0.0033483535517007113, -0.04811941087245941, 0.0688079446554184, -0.046538323163986206, -0.03846698999404907, 0.036646537482738495, -0.06330659985542297, -0.0244973786175251, 0.06278559565544128, -0.0006168063846416771, -0.010219249874353409, -0.01901366002857685, -0.013489929959177971, 0.02655153162777424, -0.057653650641441345, 0.03545270860

In [99]:
retrieved_chunks = [hit["_source"].get("text", "") for hit in response["hits"]["hits"]]
context = "\n\n".join(retrieved_chunks)


In [100]:
import os
from dotenv import load_dotenv
from ibm_watsonx_ai.foundation_models import Model

def initialize_model():
    # Load environment variables
  
    # Access environment variables
    project_id = "your_project_id"
    apikey = "your_apikey"
    model_id = "meta-llama/llama-3-3-70b-instruct"
    url = "https://us-south.ml.cloud.ibm.com"

    # Define model parameters
    parameters= {
        "decoding_method": "greedy",
        "max_new_tokens": 2000,
        "min_new_tokens": 50,
        "stop_sequences": [],
        "repetition_penalty": 1.1
    }
    # Get credentials
    credentials = {
        "url": url,
        "apikey": apikey
    }

    # Initialize and return the model
    model = Model(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id
    )
    
    return model

In [117]:

prompt = f""" Here you can use your own prompt

Data:
{context}

Query:
{query}

Answer:"""


In [ ]:
print(context)

In [103]:
model = initialize_model()  # your existing init function

In [104]:
def rag_generate(prompt, model):
    response = model.generate(prompt=prompt)
    return response['results'][0]['generated_text']

In [ ]:
output = rag_generate(prompt, model)
print(output)

In [1]:
%pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 22.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
